In [1]:
import pandas as pd

df = pd.read_csv("data/merged.csv")

- state: 환경정보
- action: SUB_MHRLS_OPRT_YN(보조기기작동여부)
    - 0 또는 1

In [2]:
environment = {
        "PFBS_NTRO_CBDX_CTRN": "관측지점실내이산화탄소농도",
        "ABSLT_HMDT": "절대습도",
        "INNER_HMDT_1": "내부습도_1",
        "INNER_HMDT_2": "내부습도_2",
        "AVE_INNER_HMDT_1_2": "평균내부습도_1_2",
        "WNDRC": "풍향",
        "WDSP": "풍속",
        "STRTN_WATER": "포화수분",
        "EXTN_SRQT": "외부일사량",
        "WATER_LACK_VL": "수분부족량",
        "EXTN_ACCMLT_QOFLG": "외부누적광량",
        "PRCPT_YN": "강수여부",
        "DWP_TPRT": "이슬점온도",
        "EXTN_TPRT": "외부온도",
        "INNER_TPRT_1": "내부온도_1",
        "INNER_TPRT_2": "내부온도_2",
        "AVE_INNER_TPRT_1_2": "평균내부온도_1_2",
}

In [3]:
df["SUB_MHRLS_OPRT_YN"] = df[["SUB_MHRLS_OPRT_YN_1","SUB_MHRLS_OPRT_YN_2", "SUB_MHRLS_OPRT_YN_3"]].apply(
    lambda x: 0 if 0 in x.values else 1, axis=1)
df["SUB_MHRLS_OPRT_YN"].value_counts()

SUB_MHRLS_OPRT_YN
0    28940
1    19444
Name: count, dtype: int64

In [5]:
var = df["SUB_MHRLS_OPRT_YN"].var()
for k in environment.keys():
    covariance = df["SUB_MHRLS_OPRT_YN"].cov(df[k])
    print(f"{environment[k]}: \t Cov is {covariance:.4f} \t Beta is {covariance/var:.4f}")

관측지점실내이산화탄소농도: 	 Cov is -10.7342 	 Beta is -44.6559
절대습도: 	 Cov is 0.5911 	 Beta is 2.4592
내부습도_1: 	 Cov is -0.5594 	 Beta is -2.3271
내부습도_2: 	 Cov is -0.2534 	 Beta is -1.0540
평균내부습도_1_2: 	 Cov is -0.4064 	 Beta is -1.6905
풍향: 	 Cov is -1.3323 	 Beta is -5.5427
풍속: 	 Cov is 0.0189 	 Beta is 0.0787
포화수분: 	 Cov is 0.9000 	 Beta is 3.7442
외부일사량: 	 Cov is 21.0940 	 Beta is 87.7546
수분부족량: 	 Cov is 0.3086 	 Beta is 1.2840
외부누적광량: 	 Cov is 65.9143 	 Beta is 274.2143
강수여부: 	 Cov is 0.0142 	 Beta is 0.0593
이슬점온도: 	 Cov is 0.6378 	 Beta is 2.6535
외부온도: 	 Cov is 2.0061 	 Beta is 8.3456
내부온도_1: 	 Cov is 0.7847 	 Beta is 3.2645
내부온도_2: 	 Cov is 0.7182 	 Beta is 2.9877
평균내부온도_1_2: 	 Cov is 0.7514 	 Beta is 3.1261


In [6]:
from util.process import MyDataset, Modeling
from util.model import LSTM

x_cols = environment.keys()
y_cols = ["SUB_MHRLS_OPRT_YN"]
data = MyDataset(df, x_cols, y_cols)
train_loader, test_loader = data.preprocessing()

In [7]:
model = Modeling(LSTM, data, criterion="crossEntropy")

Using device: cpu


In [8]:
model.train(1, train_loader, test_loader)

Epoch 1/10, Validation Loss: 0.0000


In [9]:
model.eval(test_loader, data.y_test)

explained_variance:  -0.1579
median_absolute_error:  0.9936
r2:  -103.6455
MAE:  0.988
MSE:  0.9867
RMSE:  0.9933


(array([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], dtype=float32),
 array([0.00642157, 0.00642157, 0.00642157, ..., 0.00642157, 0.00642157,
        0.00642157]))

In [ ]:
class TomatoEnv:
    def __init__(self, seq_len:int, feature_num:int, action_num:int):
        self.state_space = [[i for i in range(feature_num)] for _ in range(seq_len)]
        self.action_space = [i for i in range(action_num)]
        self.current_state = None
    
    def reset(self, initial_state):
        self.current_state = initial_state
        return self.current_state
    
    def step(self, action):
        if action == 1:
            self.current_state = min(self.current_state + 1, len(self.state_space) - 1)
        elif action == 0:
            self.current_state = max(self.current_state - 1, 0)
        # 보상 및 종료 여부 결정
        reward = 1 if self.current_state == self.grid_size - 1 else 0
        done = self.current_state == self.grid_size - 1
        
        return self.current_state, reward, done